In [98]:
from tqdm.notebook import tqdm

import datetime
import time
import requests
import re
from copy import copy
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
req = requests.get('http://www.khmcpharm.com/Lib/Modules/MediSearch/popup_01.html')
req.encoding = 'euc-kr'
dom = BeautifulSoup(req.text, 'html')

In [51]:
txt = [i.text for i in dom.select('table > tr > td')[3:]]

In [93]:
data = list()
for i in txt : 
    splitted = i.split('\n\n\n')
    for s in splitted : 
        s = re.sub("\d\s[가-힣][A-Z]", '', s)
        if '닫기' in s : 
            continue
        if not s : 
            continue
        data.append(s.replace("\n",''))
        
splitted = [i.split(" ") for i in data]
splitted = [(s[0], ','.join(s[1:]).replace(",",' '))for s in splitted]

In [95]:
dict_ = dict()

for ls in splitted : 
    id_, cls_ = ls
    dict_[id_] = cls_

In [97]:
pd.to_pickle(dict_, "data/label_mapper.pickle")

____

In [2]:
numdays = 400
base = datetime.datetime.today()
date_list = sorted([base - datetime.timedelta(days=x) for x in range(numdays)])
ymd_ls = [dt.year*10000+dt.month*100+dt.day for dt in date_list]
data_index = range(1,10+1)

In [8]:
mem = list()

for ymd in tqdm(ymd_ls) : 
    for di in data_index : 
#        time.sleep(0.5)
        
        ymd_code = ymd * 100000 + di
        url = f'https://www.health.kr/searchDrug/ajax/ajax_result_drug2.asp?drug_cd={ymd_code}'

        custom_header = {
            'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

        req = requests.get(url, headers=custom_header)
        dom = BeautifulSoup(req.text, 'html')

        is_invalid = dom.text == '[]'
        is_invalid

        if is_invalid : 
            continue
        txt = str(dom.select("body > p")[0])
        indexes = [m.start() for m in re.finditer("\{|}", txt)]
        if len(indexes) != 2 : 
            dict_form = txt
        else : 
            dict_form = txt[indexes[0]:indexes[1]]

        founds = re.findall('\"\w+\"\:\"[0-9]+\"', dict_form)
        dict_ = dict()

        for element in founds : 
            k,v = element.split(":")
            dict_[k.replace('"','')] = v.replace('"','')

        try : 
            founds = re.findall('\"atc_cd\"\:\"[A-Z0-9]+\|', dict_form)[0]
        except IndexError : 
            continue

        k,v = founds.split(":")
        dict_[k.replace('"','')] = v.replace('"','').replace("|",'')
        mem.append(dict_)        

  0%|          | 0/400 [00:00<?, ?it/s]

In [12]:
data = pd.DataFrame(mem)

In [14]:
pd.to_pickle(data, "data/label_df.pickle")